In [2]:
import json
import folium
import requests
import mimetypes
import http.client
import pandas as pd
from folium.plugins import HeatMap
from pandas.io.json import json_normalize

In [3]:
con = http.client.HTTPSConnection('api.covid19api.com')
payload = ''
headers = {}
con.request('GET','/summary', payload, headers)
res = con.getresponse()
data = res.read().decode('UTF-8')

In [7]:
covid_data = json.loads(data)
pd.json_normalize(covid_data['Countries'], sep = ',')

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date
0,Afghanistan,AF,afghanistan,81,39422,4,1466,27,32879,2020-10-06T18:36:40Z
1,Albania,AL,albania,144,14410,4,400,150,8825,2020-10-06T18:36:40Z
2,Algeria,DZ,algeria,134,52270,8,1768,94,36672,2020-10-06T18:36:40Z
3,Andorra,AD,andorra,260,2370,0,53,75,1615,2020-10-06T18:36:40Z
4,Angola,AO,angola,128,5530,4,199,14,2591,2020-10-06T18:36:40Z
...,...,...,...,...,...,...,...,...,...,...
183,Viet Nam,VN,vietnam,1,1097,0,35,2,1022,2020-10-06T18:36:40Z
184,Western Sahara,EH,western-sahara,0,10,0,1,0,8,2020-10-06T18:36:40Z
185,Yemen,YE,yemen,0,2041,1,592,3,1323,2020-10-06T18:36:40Z
186,Zambia,ZM,zambia,37,15089,1,334,119,14306,2020-10-06T18:36:40Z


In [32]:
covid_df = pd.DataFrame(covid_data['Countries'])
covid_df = covid_df.drop(columns = ['CountryCode','Slug','Date','Premium'], axis = 1)
covid_df.head()

,Country,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered
0,Afghanistan,81,39422,4,1466,27,32879
1,Albania,144,14410,4,400,150,8825
2,Algeria,134,52270,8,1768,94,36672
3,Andorra,260,2370,0,53,75,1615
4,Angola,128,5530,4,199,14,2591


In [ ]:
m = folium.Map(tiles = 'Stamen Terrain', min_zoom= 1.5)
m

In [13]:
url = 'http://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'

In [ ]:
folium.Choropleth(
    geo_data = country_shapes,
    min_zoom = 2,
    name = 'Covid-19',
    data = covid_df,
    columns = ['Country','TotalConfirmed'],
    key_on = 'feature.properties.name',
    fill_color = 'OrRd',
    nan_fill_color = 'black',
    legend_name = 'Total Confirmed Covid Cases'
).add_to(m)
m

In [33]:
covid_df.update(covid_df['TotalConfirmed'].map('Total Confirmed:{}'.format))
covid_df.update(covid_df['TotalRecovered'].map('Total Recovered:{}'.format))

In [26]:
coordinates = pd.read_csv('https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv')

In [34]:
covid_final = pd.merge(covid_df, coordinates, on = 'Country')

In [40]:
def plotDot(point):
  folium.CircleMarker(location = [point.latitude, point.longitude],
                      radius = 5, weight = 2,
                      popup = folium.Popup([point.Country, point.TotalConfirmed, point.TotalRecovered]),
                      fill_color = '000000').add_to(m)  

In [41]:
covid_final.apply(plotDot, axis = 1)
m.fit_bounds(m.get_bounds())
m